Transforming training set words to features with TF-IDF values

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import csv
import numpy as np

df=pd.read_csv('training.csv')

# cleanup_nums = {"topic": {"IRRELEVANT": 0, "ARTS CULTURE ENTERTAINMENT": 1, "BIOGRAPHIES PERSONALITIES PEOPLE": 2, "DEFENCE": 3, "DOMESTIC MARKETS": 4,
#                                   "FOREX MARKETS": 5, "HEALTH": 6, "MONEY MARKETS": 7, "SCIENCE AND TECHNOLOGY": 8, "SHARE LISTINGS": 9, "SPORTS":10 }}


# df.replace(cleanup_nums, inplace=True)

vectorizer = TfidfVectorizer()

#give each word a tf-idf value
train_features = vectorizer.fit_transform(df['article_words'])

print(train_features.shape)

y_train = df['topic']

print(y_train)


(9500, 35822)
0       FOREX MARKETS
1       MONEY MARKETS
2              SPORTS
3       FOREX MARKETS
4          IRRELEVANT
            ...      
9495          DEFENCE
9496       IRRELEVANT
9497    FOREX MARKETS
9498       IRRELEVANT
9499    FOREX MARKETS
Name: topic, Length: 9500, dtype: object


Transforming test set words to features with TF-IDF values

In [2]:
df_test=pd.read_csv('test.csv')

# cleanup_nums = {"topic": {"IRRELEVANT": 0, "ARTS CULTURE ENTERTAINMENT": 1, "BIOGRAPHIES PERSONALITIES PEOPLE": 2, "DEFENCE": 3, "DOMESTIC MARKETS": 4,
#                                   "FOREX MARKETS": 5, "HEALTH": 6, "MONEY MARKETS": 7, "SCIENCE AND TECHNOLOGY": 8, "SHARE LISTINGS": 9, "SPORTS":10 }}


# df_test.replace(cleanup_nums, inplace=True)

#give each word a tf-idf value
test_features = vectorizer.transform(df_test['article_words'])

print(test_features.shape)

y_test = df_test['topic']

print(y_test)

(500, 35822)
0          IRRELEVANT
1          IRRELEVANT
2       FOREX MARKETS
3          IRRELEVANT
4          IRRELEVANT
            ...      
495        IRRELEVANT
496            SPORTS
497     MONEY MARKETS
498    SHARE LISTINGS
499        IRRELEVANT
Name: topic, Length: 500, dtype: object


Run ML algos

Naive Bayes Rule

In [3]:
# #naive bayes rule

# from sklearn.naive_bayes import MultinomialNB
# NBclf = MultinomialNB()

# NBclf.fit(train_features, y_train)

# #gets the NB topic predicitions of the test data
# NBpredictions = NBclf.predict(test_features)

# np.mean(NBpredictions == y_test)

# #tuning

# # from sklearn.model_selection import GridSearchCV
# # gs_clf = GridSearchCV(NBclf, parameters, n_jobs=-1)
# # gs_clf = gs_clf.fit(twenty_train.data, twenty_train.target)



Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

# RFclf = RandomForestClassifier(n_estimators=100, bootstrap = True, max_features = 'sqrt')
parameters = {
                'n_estimators': [50, 150],
                'bootstrap': (True, False), 
                'max_features': ('auto', 'sqrt', 'log2', None)
             }
rf = RandomForestClassifier()
RFclf = GridSearchCV(rf, parameters)
RFclf.fit(train_features, y_train)

pd.DataFrame(RFclf.cv_results_)


In [ ]:
# Use the forest's predict method on the test data
RFpredictions = RFclf.predict(test_features)

np.mean(RFpredictions == y_test)

SVM

In [ ]:
# from sklearn.svm import SVC

# SVM_clf = SVC(kernel='linear')
# SVM_clf.fit(train_features, y_train)
# SVMpredictions = SVM_clf.predict(test_features)


# print("hello")
# np.mean(SVMpredictions == y_test)




In [ ]:
# import numpy as np
# from sklearn.linear_model import LogisticRegression

# LGR_clf = LogisticRegression(random_state=0)
# LGR_clf.fit(train_features, y_train)
# LGR_prediction = LGR_clf.predict(test_features)

# np.mean(LGR_prediction == y_test)

In [ ]:
# import numpy as np
# from sklearn import linear_model
# from sklearn.calibration import CalibratedClassifierCV
# from sklearn.model_selection import GridSearchCV

# SGD_clf = linear_model.SGDClassifier(tol=1e-2, shuffle=False, learning_rate='constant', eta0=1)
# SGD_clf.fit(train_features, y_train)

In [ ]:
# SGD_prediction = SGD_clf.predict(test_features)
# np.mean(SGD_prediction == y_test)

In [ ]:
import numpy as np
from sklearn.calibration import CalibratedClassifierCV

clf_sigmoid = CalibratedClassifierCV(RFclf, cv='prefit', method='sigmoid')
clf_sigmoid.fit(train_features, y_train)

target_names = ["ARTS CULTURE ENTERTAINMENT", "BIOGRAPHIES PERSONALITIES PEOPLE", "DEFENCE", "DOMESTIC MARKETS", "FOREX MARKETS", "HEALTH", "IRRELEVANT", "MONEY MARKETS", "SCIENCE AND TECHNOLOGY", "SHARE LISTINGS", "SPORTS"] 


pd.DataFrame(clf_sigmoid.predict_proba(test_features)*100, columns=target_names)

In [ ]:
proba=list(map(list,zip(*(clf_sigmoid.predict_proba(test_features)*100))))
tempsort=[]
psort=[[],[],[],[],[],[],[],[],[],[],[]]

for i in range(11):
    keys=list(range(9501, 10001))
    tempsort.append(sorted(list(zip(keys,proba[i])), key=lambda x: x[1], reverse=True))
    tempsort[i]=tempsort[i][:10]
    for j in tempsort[i]:
        if RFpredictions[j[0] - 9501] == target_names[i]:
            psort[i].append(j)
    display(pd.DataFrame(psort[i], columns=["Article",target_names[i]]))

Classification report

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, RFpredictions, target_names=target_names))

--------------------BELOW IS NOT SUPER NEEDED DELETE LATER-------------------------

SGD Classifier

In [ ]:
df2 = pd.read_csv("training.csv")
df2.head()

In [ ]:
df2['topic'].unique()

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(10,6))
df3 = df2[['topic','article_words']]
df3.groupby('topic').count().plot.bar(ylim=0)
plt.show()

In [ ]:
# ADJUST LATER
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder

labels = df3['topic']
text = df['article_words']

# ADJUST LATER
X_train, X_test, y_train, y_test = train_test_split(text, labels, random_state=0, test_size=0.3)

count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
tf_transformer = TfidfTransformer().fit(X_train_counts)
X_train_transformed = tf_transformer.transform(X_train_counts)

X_test_counts = count_vect.transform(X_test)
X_test_transformed = tf_transformer.transform(X_test_counts)

labels = LabelEncoder()
y_train_labels_fit = labels.fit(y_train)
y_train_lables_trf = labels.transform(y_train)

print(labels.classes_)

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV

linear_svc = linear_model.SGDClassifier(max_iter=1500, tol=1e-3)
clf = linear_svc.fit(X_train_transformed,y_train_lables_trf)

calibrated_svc = CalibratedClassifierCV(base_estimator=linear_svc,
                                        cv="prefit")

calibrated_svc.fit(X_train_transformed,y_train_lables_trf)
predicted = calibrated_svc.predict(X_test_transformed)

# Need to update
to_predict = ["I have outdated information on my credit report that I have previously disputed that has yet to be removed this information is more then seven years old and does not meet credit reporting requirements"]
p_count = count_vect.transform(to_predict)
p_tfidf = tf_transformer.transform(p_count)
print('Average accuracy on test set={}'.format(np.mean(predicted == labels.transform(y_test))))
print('Predicted probabilities of demo input string are')
print(calibrated_svc.predict_proba(p_tfidf))

In [ ]:
pd.DataFrame(calibrated_svc.predict_proba(p_tfidf)*100, columns=labels.classes_)

In [ ]:
print(y_test)